##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Carregue métricas a partir de um servidor Prometheus

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/prometheus"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/prometheus.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/prometheus.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/prometheus.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Atenção: além dos pacotes Python, este notebook usa `sudo apt-get install` para instalar pacotes de terceiros.

## Visão geral

Este tutorial carrega métricas CoreDNS a partir de um servidor [Prometheus](https://prometheus.io) em um `tf.data.Dataset` e depois usa o `tf.keras` para treinamento e inferência.

O [CoreDNS](https://github.com/coredns/coredns) é um servidor DNS com foco em descoberta de serviços e é amplamente implantado como parte do cluster de [Kubernetes](https://kubernetes.io). Por esse motivo, costuma ser monitorado com atenção por operações devops.

Este tutorial é um exemplo do que pode ser usado por devops em busca de automatizar suas operações por meio de aprendizado de máquina.

## Configuração e uso

### Instale os pacotes tensorflow-io necessários e reinicie o runtime

In [ ]:
import os

In [3]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [4]:
!pip install tensorflow-io

In [ ]:
from datetime import datetime

import tensorflow as tf
import tensorflow_io as tfio

### Instale e configure o CoreDNS e o Prometheus

Para fins de demonstração, será usado um CoreDNS local com porta `9053` aberta para receber consultas DNS e porta `9153` (padrão) aberta para expor as métricas para extração. Veja abaixo uma configuração básica de Corefile para o CoreDNS, disponível para [download](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/prometheus/Corefile):

```
.:9053 {
  prometheus
  whoami
}
```

Confira mais detalhes sobre a instalação na [documentação](https://coredns.io) do CoreDNS.


In [6]:
!curl -s -OL https://github.com/coredns/coredns/releases/download/v1.6.7/coredns_1.6.7_linux_amd64.tgz
!tar -xzf coredns_1.6.7_linux_amd64.tgz

!curl -s -OL https://raw.githubusercontent.com/tensorflow/io/master/docs/tutorials/prometheus/Corefile

!cat Corefile

.:9053 {
  prometheus
  whoami
}


In [ ]:
# Run `./coredns` as a background process.
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw('./coredns &')

A próxima etapa é configurar o servidor Prometheus e usá-lo para extrair métricas do CoreDNS expostas na porta `9153`, conforme visto acima. O arquivo `prometheus.yml` para configuração também está disponível para [download](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/prometheus/prometheus.yml):


In [8]:
!curl -s -OL https://github.com/prometheus/prometheus/releases/download/v2.15.2/prometheus-2.15.2.linux-amd64.tar.gz
!tar -xzf prometheus-2.15.2.linux-amd64.tar.gz --strip-components=1

!curl -s -OL https://raw.githubusercontent.com/tensorflow/io/master/docs/tutorials/prometheus/prometheus.yml

!cat prometheus.yml

global:
  scrape_interval:     1s
  evaluation_interval: 1s
alerting:
  alertmanagers:
  - static_configs:
    - targets:
rule_files:
scrape_configs:
- job_name: 'prometheus'
  static_configs:
  - targets: ['localhost:9090']
- job_name: "coredns"
  static_configs:
  - targets: ['localhost:9153']


In [ ]:
# Run `./prometheus` as a background process.
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw('./prometheus &')

Para mostrar alguma atividade, o comando `dig` deve ser usado para gerar algumas consultas DNS ao servidor CoreDNS configurado:

In [ ]:
!sudo apt-get install -y -qq dnsutils

In [11]:
!dig @127.0.0.1 -p 9053 demo1.example.org


; <<>> DiG 9.11.3-1ubuntu1.11-Ubuntu <<>> @127.0.0.1 -p 9053 demo1.example.org
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 53868
;; flags: qr aa rd; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 3
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
; COOKIE: 855234f1adcb7a28 (echoed)
;; QUESTION SECTION:
;demo1.example.org.		IN	A

;; ADDITIONAL SECTION:
demo1.example.org.	0	IN	A	127.0.0.1
_udp.demo1.example.org.	0	IN	SRV	0 0 45361 .

;; Query time: 0 msec
;; SERVER: 127.0.0.1#9053(127.0.0.1)
;; WHEN: Tue Mar 03 22:35:20 UTC 2020
;; MSG SIZE  rcvd: 132



In [12]:
!dig @127.0.0.1 -p 9053 demo2.example.org


; <<>> DiG 9.11.3-1ubuntu1.11-Ubuntu <<>> @127.0.0.1 -p 9053 demo2.example.org
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 53163
;; flags: qr aa rd; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 3
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
; COOKIE: f18b2ba23e13446d (echoed)
;; QUESTION SECTION:
;demo2.example.org.		IN	A

;; ADDITIONAL SECTION:
demo2.example.org.	0	IN	A	127.0.0.1
_udp.demo2.example.org.	0	IN	SRV	0 0 42194 .

;; Query time: 0 msec
;; SERVER: 127.0.0.1#9053(127.0.0.1)
;; WHEN: Tue Mar 03 22:35:21 UTC 2020
;; MSG SIZE  rcvd: 132



Agora, temos um servidor CoreDNS cujas métricas são extraídas por um servidor Prometheus, prontas para serem consumidas pelo TensorFlow.

### Crie um dataset para métricas do CoreDNS e utilize-o no TensorFlow

Crie um dataset para métricas do CoreDNS que estão disponíveis no servidor Prometheus, o que pode ser feito usando `tfio.experimental.IODataset.from_prometheus`. São necessários dois argumentos, no mínimo. `query` (consulta) é passada ao servidor Prometheus para selecionar as métricas, e `length` (tamanho) é o período que você deseja carregar no dataset.

Você pode começar com `"coredns_dns_request_count_total"` e `"5"` (segundos) para criar o dataset abaixo. Como foram enviadas duas consultas DNS anteriormente neste tutorial, esperamos que as métricas `"coredns_dns_request_count_total"` sejam `"2.0"` no fim da série temporal:

In [13]:
dataset = tfio.experimental.IODataset.from_prometheus(
      "coredns_dns_request_count_total", 5, endpoint="http://localhost:9090")


print("Dataset Spec:\n{}\n".format(dataset.element_spec))

print("CoreDNS Time Series:")
for (time, value) in dataset:
  # time is milli second, convert to data time:
  time = datetime.fromtimestamp(time // 1000)
  print("{}: {}".format(time, value['coredns']['localhost:9153']['coredns_dns_request_count_total']))

Dataset Spec:
(TensorSpec(shape=(), dtype=tf.int64, name=None), {'coredns': {'localhost:9153': {'coredns_dns_request_count_total': TensorSpec(shape=(), dtype=tf.float64, name=None)}}})

CoreDNS Time Series:
2020-03-03 22:35:17: 2.0
2020-03-03 22:35:18: 2.0
2020-03-03 22:35:19: 2.0
2020-03-03 22:35:20: 2.0
2020-03-03 22:35:21: 2.0


Vamos ver mais detalhes da especificação do dataset:

```
(
  TensorSpec(shape=(), dtype=tf.int64, name=None),
  {
    'coredns': {
      'localhost:9153': {
        'coredns_dns_request_count_total': TensorSpec(shape=(), dtype=tf.float64, name=None)
      }
    }
  }
)

```

É óbvio que o dataset consiste de uma tupla `(time, values)` (horário, valores), em que o campo `values` é um dicionário (dict) do Python, expandido para:

```
"job_name": {
  "instance_name": {
    "metric_name": value,
  },
}
```

No exemplo acima, `'coredns'` é o nome do trabalho, `'localhost:9153'` é o nome da instância, e `'coredns_dns_request_count_total'` é o nome da métrica. Observe que, dependendo da consulta Prometheus usada, é possível que diversos trabalhos/instâncias/métricas sejam retornados. Também é por esse motivo que um dicionário (dict) do Python foi usado na estrutura do dataset.

Vejamos outra consulta `"go_memstats_gc_sys_bytes"` como exemplo. Como tanto o CoreDNS quanto o Prometheus são escritos em Golang, a métrica `"go_memstats_gc_sys_bytes"` está disponível tanto para o trabalho `"coredns"` quanto para o trabalho `"prometheus"`:

Observação: pode ocorrer um erro para esta célula na primeira vez em que é executada. Execute novamente para que funcione.

In [14]:
dataset = tfio.experimental.IODataset.from_prometheus(
    "go_memstats_gc_sys_bytes", 5, endpoint="http://localhost:9090")

print("Time Series CoreDNS/Prometheus Comparision:")
for (time, value) in dataset:
  # time is milli second, convert to data time:
  time = datetime.fromtimestamp(time // 1000)
  print("{}: {}/{}".format(
      time,
      value['coredns']['localhost:9153']['go_memstats_gc_sys_bytes'],
      value['prometheus']['localhost:9090']['go_memstats_gc_sys_bytes']))

Time Series CoreDNS/Prometheus Comparision:
2020-03-03 22:35:17: 2385920.0/2775040.0
2020-03-03 22:35:18: 2385920.0/2775040.0
2020-03-03 22:35:19: 2385920.0/2775040.0
2020-03-03 22:35:20: 2385920.0/2775040.0
2020-03-03 22:35:21: 2385920.0/2775040.0


Agora, o `Dataset` criado está pronto para ser passado ao `tf.keras` diretamente para treinamento ou inferência.

## Use o dataset para treinamento do modelo

Com o dataset de métricas criado, é possível passá-lo diretamente ao `tf.keras` para fazer treinamento e inferência do modelo.

Para fins de demonstração, este tutorial usará somente um modelo LSTM bem simples, com 1 característica e 2 passos como entrada:

In [ ]:
n_steps, n_features = 2, 1
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=(n_steps, n_features)),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')


O dataset a ser usado é o valor 'go_memstats_sys_bytes' para o CoreDNS, com 10 amostras. Porém, como uma janela deslizante de `window=n_steps` e `shift=1` são formados, são necessárias amostras adicionais (para quaisquer dois elementos consecutivos, o primeiro é recebido como `x`, e o segundo é recebido como `y` para o treinamento). O total é de `10 + n_steps - 1 + 1 = 12` segundos.

O valor dos dados também é redimensionado no intervalo `[0, 1]`.

In [16]:
n_samples = 10

dataset = tfio.experimental.IODataset.from_prometheus(
    "go_memstats_sys_bytes", n_samples + n_steps - 1 + 1, endpoint="http://localhost:9090")

# take go_memstats_gc_sys_bytes from coredns job 
dataset = dataset.map(lambda _, v: v['coredns']['localhost:9153']['go_memstats_sys_bytes'])

# find the max value and scale the value to [0, 1]
v_max = dataset.reduce(tf.constant(0.0, tf.float64), tf.math.maximum)
dataset = dataset.map(lambda v: (v / v_max))

# expand the dimension by 1 to fit n_features=1
dataset = dataset.map(lambda v: tf.expand_dims(v, -1))

# take a sliding window
dataset = dataset.window(n_steps, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda d: d.batch(n_steps))


# the first value is x and the next value is y, only take 10 samples
x = dataset.take(n_samples)
y = dataset.skip(1).take(n_samples)

dataset = tf.data.Dataset.zip((x, y))

# pass the final dataset to model.fit for training
simple_lstm_model.fit(dataset.batch(1).repeat(10),  epochs=5, steps_per_epoch=10)

Train for 10 steps
Epoch 1/5
10/10 [==============================] - 2s 150ms/step - loss: 0.8484
Epoch 2/5
10/10 [==============================] - 0s 10ms/step - loss: 0.7808
Epoch 3/5
10/10 [==============================] - 0s 10ms/step - loss: 0.7102
Epoch 4/5
10/10 [==============================] - 0s 11ms/step - loss: 0.6359
Epoch 5/5
10/10 [==============================] - 0s 11ms/step - loss: 0.5572


O modelo treinado acima não é muito útil, pois o servidor CoreDNS configurado neste tutorial não tem carga de trabalho nenhuma. Porém, esse é um pipeline funcional que pode ser usado para carregar métricas de servidores de produção reais. O modelo pode ser aprimorado para resolver problemas reais de automação de devops.